In [1]:
import sqlite3
import pandas as pd
from sqlalchemy import create_engine
from numpy.testing import assert_equal, assert_array_equal

## Problem 1
Create a function `create_tables` that returns a connection to an in-memory SQLite database. This database should have a table `employee` having the following columns:

* `full_name` that cannot be null and is text
* `age` that is an integer and can be null
* `rating` that is float and cannot be null
* `remarks` that is text and can be null

In [2]:
def create_tables():
    """Create table employee and return an in-memory SQLite databas connection
   
    Returns
    -------
    SQLite db with columns full_name, age, rating and remarks
    """
    conn = sqlite3.connect(':memory:')
    cursor = conn.cursor()
    
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS employee (
        full_name TEXT NOT NULL,
        age INT,
        rating FLOAT NOT NULL,
        remarks TEXT
        )
    """)
    return cursor

In [3]:
conn = create_tables()
# Note: each RDBMS has its own way of querying the table information
res = conn.execute("PRAGMA table_info('employee')").fetchall()
conn.close()
assert_equal(len(res), 4)
assert_equal(
    [col[1]  for col in res],
    ['full_name', 'age', 'rating', 'remarks']
)

## Problem 2 
Create a function `insert_values` that saves into the table `players` in `conn` the rows given by `rows`.

In [4]:
def insert_values(conn, rows):
    """Save player stats into the table 'players'
    
    Parameters
    ----------
    conn: sqlite3.Connection 
    rows: list
        player stats
    """
    cursor = conn.cursor()
    for r in rows:
        cursor.execute("""
            INSERT INTO players VALUES (?, ?, ?, ?)
        """, r)

In [5]:
rows = [
    ['Gino B.', 101, 69, 192],
    ['Big D', 46, 23, 88],
    ['VD', 57, 18, 49],
    ['Bossti', 108, 77, 256]
]
conn = sqlite3.connect('exercises.db')
conn.execute('DELETE FROM players')
insert_values(conn, rows)
res = conn.execute('SELECT * FROM players').fetchall()
conn.close()
assert_equal(
    res,
    [('Gino B.', 101, 69, 192),
     ('Big D', 46, 23, 88),
     ('VD', 57, 18, 49),
     ('Bossti', 108, 77, 256)]
)

## Problem 3
Create a function `append_values` that appends the content of data frame `df` to the `reactions` table in `conn`.

In [6]:
def append_values(conn, df):
    """Append the contents of a dataframe to the reactions 
    table in a db.
    
    Parameters
    ----------
    conn: sqlite3.Connection 
    df: pandas DataFrame
    """
    arr = df.values
    cursor = conn.cursor()
    for r in arr:
        cursor.execute("""
            INSERT INTO reactions VALUES (?,?,?,?)
        """, r)
    # df.to_sql('reactions', conn, if_exists='append', index=False)

In [7]:
df = pd.DataFrame([
        [15, 23, 1, 0],
        [3, 98, 0, 5],
        [62, 37, 2, 3],
        [81, 46, 7, 1],
        [29, 17, 4, 2]
    ],
    columns=['post_id', 'user_id', 'up', 'down'])

conn = sqlite3.connect('exercises.db')
conn.execute('DELETE FROM reactions')
append_values(conn, df)
append_values(conn, df)
conn.close()

## Problem 4
Create a function `read_table` that reads the sqlite database file `db` and returns the contents of the `transactions` table as a `pandas` `DataFrame`.

In [8]:
def read_table(db):
    """Read transactions table from database and return a pandas DataFrame
    
    Parameters
    ----------
    db: str
        relative filepath of database file
    
    Returns
    -------
    pandas DataFrame
    """
    engine = create_engine('sqlite:///' + db)
    with engine.connect() as conn:
        df = pd.read_sql('SELECT * FROM transactions', conn)
    return df

In [9]:
df_trans = read_table('exercises.db')
assert_equal(type(df_trans), pd.DataFrame)
assert_equal(df_trans.shape, (541909, 8))
assert_array_equal(
    df_trans.columns.tolist(),
    ['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
     'UnitPrice', 'CustomerID', 'Country']
)
assert_equal(
    df_trans.iloc[:5].to_numpy().tolist(),
    [['536365',  '85123A',  'WHITE HANGING HEART T-LIGHT HOLDER',  6,  
      '01/12/2010 8:26',  2.55,  17850.0,  'United Kingdom'], 
     ['536365',  '71053',  'WHITE METAL LANTERN',  6,  '01/12/2010 8:26',  
      3.39,  17850.0,  'United Kingdom'], 
     ['536365',  '84406B',  'CREAM CUPID HEARTS COAT HANGER',  8,  
      '01/12/2010 8:26',  2.75,  17850.0,  'United Kingdom'], 
     ['536365',  '84029G',  'KNITTED UNION FLAG HOT WATER BOTTLE',  6,  
      '01/12/2010 8:26',  3.39,  17850.0,  'United Kingdom'], 
     ['536365',  '84029E',  'RED WOOLLY HOTTIE WHITE HEART.',  6,  
      '01/12/2010 8:26',  3.39,  17850.0,  'United Kingdom']]
)

## Problem 5
Create a function `stocks_more_than_5` that reads the sqlite database file `db` and returns the rows of the `transactions` table with `Quantity` greater than 5, as a `pandas` `DataFrame`. Return only the columns `StockCode`, `Description` and `Quantity`. Use only a single SQL statement to answer this problem.

In [10]:
def stocks_more_than_5(db):
    """Return a pandas Dataframe with quantity column values greater than 5
    
    Parameters
    ----------
    db: SQLite
    
    Returns
    -------
    pandas DataFrame
    """
    engine = create_engine('sqlite:///' + db)
    with engine.connect() as conn:
        df = pd.read_sql("""
            SELECT StockCode, Description, Quantity
            FROM transactions
            WHERE Quantity > 5""", conn)
    return df

In [11]:
df_stocks = stocks_more_than_5('exercises.db')
assert_equal(type(df_stocks), pd.DataFrame)
assert_equal(df_stocks.shape, (213867, 3))
assert_array_equal(
    df_stocks.columns,
    ['StockCode', 'Description', 'Quantity']
)
assert_equal(
    df_stocks.iloc[:5].to_numpy().tolist(),
    [['85123A', 'WHITE HANGING HEART T-LIGHT HOLDER', 6],
     ['71053', 'WHITE METAL LANTERN', 6],
     ['84406B', 'CREAM CUPID HEARTS COAT HANGER', 8],
     ['84029G', 'KNITTED UNION FLAG HOT WATER BOTTLE', 6],
     ['84029E', 'RED WOOLLY HOTTIE WHITE HEART.', 6]]
)

## Problem 6
Create a function `get_invoices` that reads the sqlite database file `db`. The rows of the `transactions` table is grouped by `InvoiceNo` and a `pandas` `DataFrame` is returned with the following columns:
* `InvoiceNo`: `InvoiceNo` of the group
* `ItemCount`: Number of rows in the group
* `TotalQuantity`: Total `Quantity` in the group

The rows are sorted by decreasing `ItemCount`.

Your answer should be a single SQL statement that will perform all of the required operations.

In [12]:
def get_invoices(db):
    """
    Group the transactions by InvoiceNo and return a pandas DataFrame
    with columns InvoiceNo, ItemCount and TotalQuantity.
    
    Parameters
    ----------
    db: str
        relative filepath of database file
    
    Returns
    -------
    pandas DataFrame
    """
    engine = create_engine('sqlite:///' + db)
    with engine.connect() as conn:
        df = pd.read_sql("""
        SELECT 
            InvoiceNo, 
            COUNT(InvoiceNo) AS ItemCount,
            SUM(Quantity) AS TotalQuantity
        FROM transactions
        GROUP BY InvoiceNo
        ORDER BY ItemCount DESC""", conn)
    return df

In [13]:
df_invoices = get_invoices('exercises.db')
assert_equal(type(df_invoices), pd.DataFrame)
assert_equal(df_invoices.shape, (25900, 3))
assert_equal(
    df_invoices.columns.tolist(),
    ['InvoiceNo', 'ItemCount', 'TotalQuantity']
)
assert_equal(
    df_invoices.iloc[:10].to_numpy().tolist(),
    [['573585', 1114, 5198],
     ['581219', 749, 2151],
     ['581492', 731, 2011],
     ['580729', 721, 2456],
     ['558475', 705, 4137],
     ['579777', 687, 1763],
     ['581217', 676, 1853],
     ['537434', 675, 1869],
     ['580730', 662, 1858],
     ['538071', 652, 1643]]
)

## Problem 7
Create a function `white_department` that accepts a SQLAlchemy connection and returns a `pandas` `Series`. The index of the series is the `department_id` and the values are the number of rows in `products`, having `product_name` that starts with the case-sensitive word `White`, that belongs to that `department_id`. Sort by decreasing number of rows. Your answer should be a single SQL statement that will perform all of the required operations. 

In [14]:
def white_department(conn):
    """Return a pandas Series of the number of rows in products that 
    have 'White' in the product name.
    
    Parameters
    ----------
    conn: connection to SQLite db
    
    Returns
    -------
    pandas Series 
    """
    engine = create_engine('sqlite:///exercises.db')
    with engine.connect() as conn:
        ser = pd.read_sql("""
        SELECT department_id, product_name, COUNT(*) AS white
        FROM products
        WHERE LOWER(product_name) LIKE 'white %'
        GROUP BY department_id
        ORDER BY white DESC
        """, conn, index_col='department_id')['white']
    return ser    

In [15]:
engine = create_engine('sqlite:///exercises.db')
with engine.connect() as conn:
    df = pd.read_sql("""
    SELECT department_id, product_name, COUNT(*) AS white
    FROM products
    WHERE LOWER(product_name) LIKE 'white %'
    GROUP BY department_id
    ORDER BY white DESC
    """, conn, index_col='department_id')
df

,product_name,white
department_id,,
13,White Writing Gel,42
19,White Cheddar Popcorn,40
17,White Revive Laundry Stain Remover 45 Loads,28
7,White Tea Lavender Probiotic Kefir Water No. 2,28
4,White Pearl Onions,20
15,White Chicken Chili,18
3,White Hot Dog Rolls,18
1,White Chocolate Mint Ice Cream,16
9,White Cheddar Macaroni & Cheese,15


In [16]:
engine = create_engine('sqlite:///exercises.db')
with engine.connect() as conn:
    df_white = white_department(conn)
assert isinstance(df_white, pd.Series)
assert_equal(len(df_white), 18)
assert_equal(
    df_white.index[:10].tolist(), 
    [13, 19, 17, 7, 4, 15, 3, 1, 9, 11])
assert_equal(
    df_white[:10].tolist(),
    [42, 40, 28, 28, 20, 18, 18, 16, 15, 12]
)

## Problem 8
Create a function `count_aisle_products` that accepts a SQLAlchemy connection and returns a `pandas` `DataFrame`. It has the following columns:
* `aisle_id` of product in `products`
* `aisle` from `aisles` based on `aisle_id`
* `product_count` which is the number of products in `products` with `aisle_id`

Only aisles with `product_count` less than 100 should be included. Sort by increasing `product_count`. Your answer should be a single SQL statement that will perform all of the required operations.

In [17]:
def count_aisle_products(conn):
    """Return a DataFrame that contains aisles with product count less
    than 100.
    
    Parameters
    ----------
    conn: connection to SQLite db
    
    Returns
    -------
    pandas DataFrame
    """
    with engine.connect() as conn:
        df = pd.read_sql("""
        SELECT p.aisle_id, a.aisle, COUNT(*) as product_count
        FROM products AS p
        JOIN aisles AS a
        ON p.aisle_id = a.aisle_id
        GROUP BY p.aisle_id
        HAVING product_count < 100
        ORDER BY product_count
        """, conn)
    return df

In [18]:
engine = create_engine('sqlite:///exercises.db')
with engine.connect() as conn:
    df = pd.read_sql("""
    SELECT *
    FROM products
    """, conn)
df

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13
...,...,...,...,...
49683,49684,"Vodka, Triple Distilled, Twist of Vanilla",124,5
49684,49685,En Croute Roast Hazelnut Cranberry,42,1
49685,49686,Artisan Baguette,112,3
49686,49687,Smartblend Healthy Metabolism Dry Cat Food,41,8


In [19]:
with engine.connect() as conn:
    df = pd.read_sql("""
    SELECT *
    FROM aisles
    """, conn)
df

,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation
...,...,...
129,130,hot cereal pancake mixes
130,131,dry pasta
131,132,beauty
132,133,muscles joints pain relief


In [20]:
engine = create_engine('sqlite:///exercises.db')
with engine.connect() as conn:
    df_aisle = count_aisle_products(conn)
assert_equal(type(df_aisle), pd.DataFrame)
assert_equal(df_aisle.shape, (15, 3))
assert_equal(
    df_aisle.columns.tolist(),
    ['aisle_id', 'aisle', 'product_count']
)
assert_equal(
    df_aisle.iloc[:5].to_numpy().tolist(),
    [[18, 'bulk dried fruits vegetables', 12],
     [68, 'bulk grains rice dried goods', 26],
     [32, 'packaged produce', 32],
     [82, 'baby accessories', 44],
     [113, 'frozen juice', 47]]
)

## Problem 9
Create a function `concat_cols` that accepts a SQLAlchemy connection and sets the value of `col3` for rows with even `col2` to the concatenation of `col` and `col2`. Your answer should be a single SQL statement that will perform all of the required operations.

In [21]:
def concat_cols(conn):
    """Concatenates col1 and col2 values to col3 where col2 values are even
    
    Parameters
    ----------
    conn: connection to SQLite db
    """
    conn.execute("""UPDATE cols
    SET col3 = col1||col2
    WHERE col2%2 = 0
    """)

In [22]:
with create_engine('sqlite:///').connect() as conn:
    conn.execute("""
    CREATE TABLE cols (
        col1 TEXT,
        col2 INT,
        col3 TEXT
    )""")
    conn.execute("""
    INSERT INTO cols (col1, col2)
    VALUES ('a', 1), ('b', 2), ('c', 3), ('d', 4), ('e', 5)
    """)
    concat_cols(conn)
    df_concat = pd.read_sql('cols', conn)
assert_equal(df_concat.shape, (5, 3))
assert_array_equal(df_concat.columns, ['col1', 'col2', 'col3'])

## Problem 10
Create a function `del_row` that accepts a SQLAlchemy connection and deletes rows with a `col1` value of `d`. Your answer should be a single SQL statement that will perform all of the required operations.

In [23]:
def del_row(conn):
    """Delete row in a db with col1 value of d
    
    Parameters
    ----------
    conn: connection to SQLite db
    """
    conn.execute("""DELETE FROM cols
    WHERE col1 LIKE 'd'""")

In [24]:
with create_engine('sqlite:///').connect() as conn:
    conn.execute("""
    CREATE TABLE cols (
        col1 TEXT,
        col2 INT
    )""")
    conn.execute("""
    INSERT INTO cols
    VALUES ('a', 1), ('b', 2), ('c', 3), ('d', 4), ('e', 5)
    """)
    conn.execute("""DELETE FROM cols
    WHERE col1 LIKE 'd'""")
    df_del = pd.read_sql('cols', conn)
df_del

,col1,col2
0,a,1
1,b,2
2,c,3
3,e,5


In [25]:
with create_engine('sqlite:///').connect() as conn:
    conn.execute("""
    CREATE TABLE cols (
        col1 TEXT,
        col2 INT
    )""")
    conn.execute("""
    INSERT INTO cols
    VALUES ('a', 1), ('b', 2), ('c', 3), ('d', 4), ('e', 5)
    """)
    del_row(conn)
    df_del = pd.read_sql('cols', conn)
assert_equal(df_del.shape, (4, 2))
assert_array_equal(df_del.columns, ['col1', 'col2'])